### Model SVM

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import metrics

df=pd.read_csv('pre_title_baru.csv')
# df=pd.read_csv('pre_title_steaming.csv')
df.head(10)

,title,url_berita,sentimen
0,kapolri bicara sumpah pemuda demiindonesia,https://20.detik.com/demi-indonesia/20231027-2...,1
1,greysia polii ajak anak muda melangkah raih pr...,https://20.detik.com/demi-indonesia/20231027-2...,1
2,pan jakcloth pesta anak nongkrong sambangi bar...,https://20.detik.com/advertorial/20231109-2311...,1
3,strategi sleman dorong ekonomi pakai batik int...,https://20.detik.com/blak-blakan/20231204-2312...,1
4,iringiringan pengaw presiden joe biden ditabra...,https://20.detik.com/detikjabar/20231218-23121...,1
5,bertemu habib rizieq cak imin bantah baha duku...,https://20.detik.com/detikupdate/20231001-2310...,1
6,erick thohir ajak muda mager indonesia,https://20.detik.com/demi-indonesia/20231027-2...,1
7,arahan megawati rakerna iv pdip turun rakyatme...,https://20.detik.com/detikupdate/20231001-2310...,1
8,pdip ganjar muliakan petani nelayan presiden,https://20.detik.com/detikupdate/20231001-2310...,1
9,ani bertemu alumni itb baha metod ilmiah menga...,https://20.detik.com/detikupdate/20231001-2310...,1


In [3]:
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'
df['sentimen'] = df['sentimen'].map(map_sentiment)


In [4]:
X = df['title']
y = df['sentimen']

In [5]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
    test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y

norm_train_corpus, norm_test_corpus, train_labels, test_labels = prepare_datasets(X,y,test_data_proportion=0.3)

In [6]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix
def tfidf_extractor(corpus, ngram_range=(1,2)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [7]:
#mengunakan model TF-IDF untuk mengekstraksi feature
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

In [8]:
from sklearn.svm import SVC

In [9]:
# Create SVM model
svm_model = SVC(kernel='linear', random_state=42)
# Train SVM model
svm_model.fit(tfidf_train_features, train_labels)

SVC(kernel='linear', random_state=42)

In [10]:
# Predict
svm_pred = svm_model.predict(tfidf_test_features)

# Accuracy
svm_accuracy = accuracy_score(test_labels, svm_pred)
print("Accuracy:", svm_accuracy)

Accuracy: 0.6923232323232323


In [11]:
print(metrics.classification_report(test_labels, svm_pred))



              precision    recall  f1-score   support

     negatif       0.67      0.68      0.68      1637
      netral       0.70      0.66      0.68      1648
     positif       0.70      0.73      0.72      1665

    accuracy                           0.69      4950
   macro avg       0.69      0.69      0.69      4950
weighted avg       0.69      0.69      0.69      4950



In [12]:
test = ('Jeka Saragih On Fire Menatap Debut di UFC')
line = tfidf_vectorizer.transform([test])
line.shape
test_result = svm_model.predict(line)
print("Hasil Prediksi:", test_result) 


Hasil Prediksi: ['negatif']
